<a href="https://colab.research.google.com/github/ahabibz/Rade-Yab/blob/main/Rade_Yab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import arabic_reshaper
from bidi.algorithm import get_display
from PIL import Image, ImageFont, ImageDraw


INCH = 25.4
DPI = 300
QUALITY = 90


FIRST_PAGE_TITLE = 'آرشيو سوالات المپياد فيزيک رده ب'
AUTHOR = 'Alireza Habibzade'
LEGAL_NOTICE = '''
PDF created by "Rade-Yab" (Rade-B Question-Bank Database)

Meant only to be used by Allame Helli High School
Copyright (©) 2020-2021 All Rights Reserved. '''




def addtext(img, txt):
    size = img.size
    font = ImageFont.truetype("Fcour.ttf", round(size[0]*0.02))
    imgdraw = ImageDraw.Draw(img)
    imgdraw.text((size[0]*0.015, size[0]*0.01), txt, fill = 'BLACK',
                 font = font, anchor = None, spacing = 4, align = 'left',
                 direction = None, features = None, language=None,
                 stroke_width = round(size[0]*0.005), 
                 stroke_fill = 'WHITE', embedded_color = False)
    return img
    


file_name = input('file name: ')

sheets_input = input('\nCopy and paste the list from sheets: ')
Qlist = sheets_input.replace('\t','/').splitlines()
print(Qlist)


imglist = []
error_count = 0

for q in Qlist:
    try:
        img = Image.open(f'{q}.jpg')
        img = img.convert('RGB')
        imglist.append(addtext(img, f'SS/{q}'))
    except FileNotFoundError:
        i = 1
        while True:
            try:
                img = Image.open(f'{q}-{i}.jpg')
                img = img.convert('RGB')
                imglist.append(addtext(img, f'SS/{q} page {i}'))
                i += 1               
            except FileNotFoundError:
                if i == 1:
                    print("Couldn't find", q)
                    error_count += 1
                break

if error_count:
    print(f"\nCouldn't find {error_count} questions in total of {len(Qlist)}")
else:
    print(f"\nAll {len(Qlist)} questions had been found successfully")


size = (round(210*DPI/INCH), round(297*DPI/INCH))
first_page = Image.new('RGB', size, 'WHITE')

font = ImageFont.truetype("calibrib.ttf", round(0.4*DPI))
fontb = ImageFont.truetype("calibrib.ttf", round(12*DPI/len(file_name)))
fontc = ImageFont.truetype("Fcour.ttf", round(0.15*DPI))

imgdraw = ImageDraw.Draw(first_page)

imgdraw.text((size[0]*0.3, size[0]*0.15), 
             get_display(arabic_reshaper.reshape(FIRST_PAGE_TITLE)),
             fill='BLACK', font=font, align='right',
             stroke_width=round(DPI*0.5/len(file_name)), stroke_fill='WHITE')

imgdraw.text((size[0]*0.15, size[0]*0.32),
             get_display(arabic_reshaper.reshape(file_name)),
             fill='WHITE', font=fontb, align='right',
             stroke_width=round(DPI*0.6/len(file_name)), stroke_fill='BLACK')

imgdraw.multiline_text((size[0]*0.02, size[1]*0.927),
                       LEGAL_NOTICE, fill='BLACK', font=fontc, align='left')


first_page.save('PDF/' + file_name + '.pdf', save_all=True, quality=QUALITY,
          append_images=imglist, resolution=DPI, author=AUTHOR)
print('*** PDF created successfully ***')

for i in range(len(imglist)):
    imglist[i].save(f'IMG/{i}.jpg', quality=QUALITY, resolution=DPI)
print('*** Images saved successfully ***')

